In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import json
import os
import requests
from bs4 import BeautifulSoup

In [3]:
def get_scores(day):
    url = f"https://www.ncaa.com/scoreboard/soccer-men/d1/2024/{day}/all-conf"

    # Send a request to fetch the HTML content of the page
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        print("Error fetching the page.")
        return []

    # Create a BeautifulSoup object
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all game pods with the final status
    games = soup.find_all('div', class_='gamePod gamePod-type-game status-final')

    scores = []  # List to hold score results

    # Iterate through each game and extract relevant information
    for game in games:
        teams = game.find_all('li')

        if len(teams) < 2:  # Ensure there are at least two teams
            continue

        # Away team information (always the first team listed)
        away_team_name = teams[0].find('span', class_='gamePod-game-team-name').text.strip()
        away_team_score = teams[0].find('span', class_='gamePod-game-team-score').text.strip()

        # Home team information (always the second team listed)
        home_team_name = teams[1].find('span', class_='gamePod-game-team-name').text.strip()
        home_team_score = teams[1].find('span', class_='gamePod-game-team-score').text.strip()
        
        # Append the result to the scores list
        scores.append({
            'home_team': home_team_name,
            'home_team_score': home_team_score,
            'away_team': away_team_name,
            'away_team_score': away_team_score,
        })

    return scores  # Return the scores list


In [4]:
start_date = '2024-08-22'
end_date = '2024-09-25'

# Generate a date range
date_range = pd.date_range(start=start_date, end=end_date)

# Extract month/day for each date
time_range = date_range.strftime('%m/%d').tolist()

In [5]:
dfs = []
for day in time_range:
    scores = get_scores(day)
    
    scores_df = pd.DataFrame(scores)
    
    dfs.append(scores_df)
    
dfs = pd.concat(dfs, ignore_index=True)
#dfs.to_csv(rf'results/ncaa_scores_{end_date}.csv', index=False)

In [6]:
scores

[{'home_team': 'Marshall',
  'home_team_score': '1',
  'away_team': 'VCU',
  'away_team_score': '0'},
 {'home_team': 'SMU',
  'home_team_score': '1',
  'away_team': 'FIU',
  'away_team_score': '1'},
 {'home_team': 'Southern Ind.',
  'home_team_score': '1',
  'away_team': 'Rose-Hulman',
  'away_team_score': '0'},
 {'home_team': 'Utah Valley',
  'home_team_score': '0',
  'away_team': 'Oregon St.',
  'away_team_score': '1'},
 {'home_team': 'California Baptist',
  'home_team_score': '0',
  'away_team': 'UC Riverside',
  'away_team_score': '0'},
 {'home_team': 'Pacific',
  'home_team_score': '1',
  'away_team': 'CSU Bakersfield',
  'away_team_score': '2'}]

In [7]:
conferences = pd.read_csv('teams_conferences.csv')
conferences = conferences[['School', 'Conference']]

In [8]:
conferences.columns = ['Team', 'Conference']

In [9]:
# First, merge to get the home_team_conference
dfs = dfs.merge(conferences, left_on='home_team', right_on='Team', how='left')
dfs = dfs.rename(columns={'Conference': 'home_team_conference'}).drop('Team', axis=1)

# Then, merge to get the away_team_conference
dfs = dfs.merge(conferences, left_on='away_team', right_on='Team', how='left')
dfs = dfs.rename(columns={'Conference': 'away_team_conference'}).drop('Team', axis=1)

# Now dfs will have the columns ['home_team_conference', 'away_team_conference']


In [12]:
dfs.to_excel(rf'results/ncaa_scores_{end_date}.xlsx', index=False)